In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('train.csv', encoding='ISO-8859-1', index_col='Id')
data

,Hotel_name,Review_Title,Review_Text,Rating
Id,,,,
0,Park Hyatt,Refuge in Chennai,Excellent room and exercise facility. All arou...,80.0
1,Hilton Chennai,Hilton Chennai,Very comfortable and felt safe. \r\nStaff were...,100.0
2,The Royal Regency,No worth the rating shown in websites. Pricing...,Not worth the rating shown. Service is not goo...,71.0
3,Rivera,Good stay,"First of all nice & courteous staff, only one ...",86.0
4,Park Hyatt,Needs improvement,Overall ambience of the hotel is very good. In...,86.0
...,...,...,...,...
2346,Hyatt Regency Chennai,NaN,Most impressive service by staff in all areas....,80.0
2347,New Woodlands,Homely villa,New woodlands chennai which gave me a homely e...,71.0
2348,Samudra Residency,Nice accommodation and facilities,Awesome I liked the neatness and maintenance. ...,100.0


In [ ]:
import numpy as np
np.unique(data.Rating)

array([ 20.,  29.,  40.,  43.,  57.,  60.,  65.,  71.,  80.,  86., 100.])

In [ ]:
import collections

collections.Counter(data.Rating)

Counter({20.0: 88,
         29.0: 69,
         40.0: 88,
         43.0: 87,
         57.0: 144,
         60.0: 172,
         65.0: 163,
         71.0: 327,
         80.0: 307,
         86.0: 465,
         100.0: 441})

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from nltk.corpus import wordnet

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='english')

In [ ]:
def clean_text(text):
    # Make lowercase
    text = text.lower()

    #Tokenizing with punctuation
    text = [word.strip(string.punctuation) for word in text.split(" ")]

    #Delete toneks with digits
    text = [word for word in text if not any(c.isdigit() for c in word)]

    #Delete stopwords
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    
    text = [t for t in text if len(t) > 0]
    pos_tags = pos_tag(text)
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    text = [t for t in text if len(t) > 1]
    text = " ".join(text)
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    return(text)

# reviews_df["review_clean"] = reviews_df["review"].apply(lambda x: clean_text(x))

Посмотрим на результат работы:

In [ ]:
data.loc[0, 'Review_Text']

'Excellent room and exercise facility. All around atmosphere was calm and comfortable. Main dining room offers both excellent food and service. Avoid Flying Elephant restaurant, stick to the main dining room.'

In [ ]:
clean_text(data.loc[0, 'Review_Text'])

'excel room exercis facil around atmospher calm comfort main dine room offer excel food servic avoid fli eleph restaur stick main dine room'

Создадим новый столбец:

In [ ]:
data['Review_clean'] = data["Review_Text"].apply(lambda x: clean_text(x))
data

,Hotel_name,Review_Title,Review_Text,Rating,Review_clean
Id,,,,,
0,Park Hyatt,Refuge in Chennai,Excellent room and exercise facility. All arou...,80.0,excel room exercis facil around atmospher calm...
1,Hilton Chennai,Hilton Chennai,Very comfortable and felt safe. \r\nStaff were...,100.0,comfort felt safe staff help respect breakfast...
2,The Royal Regency,No worth the rating shown in websites. Pricing...,Not worth the rating shown. Service is not goo...,71.0,worth rate show servic good room well maintain...
3,Rivera,Good stay,"First of all nice & courteous staff, only one ...",86.0,first nice courteous staff one con stay time c...
4,Park Hyatt,Needs improvement,Overall ambience of the hotel is very good. In...,86.0,overal ambienc hotel good room facil need impr...
...,...,...,...,...,...
2346,Hyatt Regency Chennai,NaN,Most impressive service by staff in all areas....,80.0,impress servic staff area good restaur fit cen...
2347,New Woodlands,Homely villa,New woodlands chennai which gave me a homely e...,71.0,new woodland chennai give home experi luxuri t...
2348,Samudra Residency,Nice accommodation and facilities,Awesome I liked the neatness and maintenance. ...,100.0,awesom like neat mainten facil reason price ov...


Попробуем определить настроение обзора:

In [ ]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
data["sentiments"] = data["Review_clean"].apply(lambda x: sid.polarity_scores(x))

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
data['sentiments_neg'] = data['sentiments'].apply(lambda x: x['neg'])
data['sentiments_neu'] = data['sentiments'].apply(lambda x: x['neu'])
data['sentiments_pos'] = data['sentiments'].apply(lambda x: x['pos'])
data['sentiments_compound'] = data['sentiments'].apply(lambda x: x['compound'])
data

,Hotel_name,Review_Title,Review_Text,Rating,Review_clean,sentiments,sentiments_neg,sentiments_neu,sentiments_pos,sentiments_compound
Id,,,,,,,,,,
0,Park Hyatt,Refuge in Chennai,Excellent room and exercise facility. All arou...,80.0,excel room exercis facil around atmospher calm...,"{'neg': 0.071, 'neu': 0.581, 'pos': 0.348, 'co...",0.071,0.581,0.348,0.8225
1,Hilton Chennai,Hilton Chennai,Very comfortable and felt safe. \r\nStaff were...,100.0,comfort felt safe staff help respect breakfast...,"{'neg': 0.0, 'neu': 0.537, 'pos': 0.463, 'comp...",0.000,0.537,0.463,0.8807
2,The Royal Regency,No worth the rating shown in websites. Pricing...,Not worth the rating shown. Service is not goo...,71.0,worth rate show servic good room well maintain...,"{'neg': 0.077, 'neu': 0.496, 'pos': 0.427, 'co...",0.077,0.496,0.427,0.9022
3,Rivera,Good stay,"First of all nice & courteous staff, only one ...",86.0,first nice courteous staff one con stay time c...,"{'neg': 0.094, 'neu': 0.59, 'pos': 0.316, 'com...",0.094,0.590,0.316,0.7506
4,Park Hyatt,Needs improvement,Overall ambience of the hotel is very good. In...,86.0,overal ambienc hotel good room facil need impr...,"{'neg': 0.0, 'neu': 0.884, 'pos': 0.116, 'comp...",0.000,0.884,0.116,0.4404
...,...,...,...,...,...,...,...,...,...,...
2346,Hyatt Regency Chennai,NaN,Most impressive service by staff in all areas....,80.0,impress servic staff area good restaur fit cen...,"{'neg': 0.0, 'neu': 0.57, 'pos': 0.43, 'compou...",0.000,0.570,0.430,0.9678
2347,New Woodlands,Homely villa,New woodlands chennai which gave me a homely e...,71.0,new woodland chennai give home experi luxuri t...,"{'neg': 0.0, 'neu': 0.811, 'pos': 0.189, 'comp...",0.000,0.811,0.189,0.7717
2348,Samudra Residency,Nice accommodation and facilities,Awesome I liked the neatness and maintenance. ...,100.0,awesom like neat mainten facil reason price ov...,"{'neg': 0.0, 'neu': 0.515, 'pos': 0.485, 'comp...",0.000,0.515,0.485,0.9042


Попробуем сделать dummies для названия отеля

In [ ]:
dum = pd.get_dummies(data.Hotel_name)
dum.head()

,Aadithya,Abu sarovar portico ex Abu Palace,Accord Metropolitan,Akash Inn,Akshay Inn,Aloft Chennai Omr - It Expressway,Ambica Empire,Ammu Palace,Ampa Skywalk,Annsun Boutique,Apollo Serviced Apartments,Apple Residency,Aryaas,Asiana Place,Atchaya,Avnb Towers,Azeez Avenue,BKR Convention Centre,BKR Grand,Beach Park Resort,Bell Hotel Chennai,Benzz Park,Beverly Hotel,Bhimaas,Bluten by Traditions Inn,Brownstar Boutique Hotel,Buhari Royale,Chandra Park,Chennai Palace,Clarion Hotel President,Cloud Nine Serviced Apartments,Courtyard Chennai India,Crescent Hospitality Chennai,Crescent Park,Crystal Residency,Deccan Plaza,Dee Cee Manor,Dolphin Park,E Hotel,Ekaana Cottages,...,The Chariot,The Checkers,The Gateway IT Expressway,The Iris,The Kings,The Leela Palace,The Lotus,The Lotus Sameera,The Lotus Serviced Apartments,The Mount Manor,The Park Chennai,The Pride,The Raintree,"The Raintree, St. Mary's Road",The Residency Chennai,The Residency Towers,The Royal Regency,The Spin,The Spring,The Vijay Park-Arumbakkam,The Westin Chennai Velachery,Thulashi Park,Treebo Angson,Treebo Elan,Treebo Grand Marena,Treebo Hive,Treebo J's Five Two Boutique,Treebo Nestlay Casa,Treebo Saravana Boutique Inn,Tri Sea Residency,Trident Chennai,Turyaa Chennai,VGP Golden Beach Resort,Vestin Park,Victoria,Yashwanth Residency,Zama Lodge,Zone By The Park Orr Chennai,ibis Chennai City Centre,ibis Chennai Sipcot
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
data = data.join(dum)
data

,Hotel_name,Review_Title,Review_Text,Rating,Review_clean,sentiments,sentiments_neg,sentiments_neu,sentiments_pos,sentiments_compound,Aadithya,Abu sarovar portico ex Abu Palace,Accord Metropolitan,Akash Inn,Akshay Inn,Aloft Chennai Omr - It Expressway,Ambica Empire,Ammu Palace,Ampa Skywalk,Annsun Boutique,Apollo Serviced Apartments,Apple Residency,Aryaas,Asiana Place,Atchaya,Avnb Towers,Azeez Avenue,BKR Convention Centre,BKR Grand,Beach Park Resort,Bell Hotel Chennai,Benzz Park,Beverly Hotel,Bhimaas,Bluten by Traditions Inn,Brownstar Boutique Hotel,Buhari Royale,Chandra Park,Chennai Palace,Clarion Hotel President,...,The Chariot,The Checkers,The Gateway IT Expressway,The Iris,The Kings,The Leela Palace,The Lotus,The Lotus Sameera,The Lotus Serviced Apartments,The Mount Manor,The Park Chennai,The Pride,The Raintree,"The Raintree, St. Mary's Road",The Residency Chennai,The Residency Towers,The Royal Regency,The Spin,The Spring,The Vijay Park-Arumbakkam,The Westin Chennai Velachery,Thulashi Park,Treebo Angson,Treebo Elan,Treebo Grand Marena,Treebo Hive,Treebo J's Five Two Boutique,Treebo Nestlay Casa,Treebo Saravana Boutique Inn,Tri Sea Residency,Trident Chennai,Turyaa Chennai,VGP Golden Beach Resort,Vestin Park,Victoria,Yashwanth Residency,Zama Lodge,Zone By The Park Orr Chennai,ibis Chennai City Centre,ibis Chennai Sipcot
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,Park Hyatt,Refuge in Chennai,Excellent room and exercise facility. All arou...,80.0,excel room exercis facil around atmospher calm...,"{'neg': 0.071, 'neu': 0.581, 'pos': 0.348, 'co...",0.071,0.581,0.348,0.8225,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hilton Chennai,Hilton Chennai,Very comfortable and felt safe. \r\nStaff were...,100.0,comfort felt safe staff help respect breakfast...,"{'neg': 0.0, 'neu': 0.537, 'pos': 0.463, 'comp...",0.000,0.537,0.463,0.8807,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,The Royal Regency,No worth the rating shown in websites. Pricing...,Not worth the rating shown. Service is not goo...,71.0,worth rate show servic good room well maintain...,"{'neg': 0.077, 'neu': 0.496, 'pos': 0.427, 'co...",0.077,0.496,0.427,0.9022,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Rivera,Good stay,"First of all nice & courteous staff, only one ...",86.0,first nice courteous staff one con stay time c...,"{'neg': 0.094, 'neu': 0.59, 'pos': 0.316, 'com...",0.094,0.590,0.316,0.7506,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Park Hyatt,Needs improvement,Overall ambience of the hotel is very good. In...,86.0,overal ambienc hotel good room facil need impr...,"{'neg': 0.0, 'neu': 0.884, 'pos': 0.116, 'comp...",0.000,0.884,0.116,0.4404,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2346,Hyatt Regency Chennai,NaN,Most impressive service by staff in all areas....,80.0,impress servic staff area good restaur fit cen...,"{'neg': 0.0, 'neu': 0.57, 'pos': 0.43, 'compou...",0.000,0.570,0.430,0.9678,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2347,New Woodlands,Homely villa,New woodlands chennai which gave me a hom

In [ ]:
data.fillna(value='Average', inplace=True)
data

,Hotel_name,Review_Title,Review_Text,Rating,Review_clean,sentiments,sentiments_neg,sentiments_neu,sentiments_pos,sentiments_compound,Aadithya,Abu sarovar portico ex Abu Palace,Accord Metropolitan,Akash Inn,Akshay Inn,Aloft Chennai Omr - It Expressway,Ambica Empire,Ammu Palace,Ampa Skywalk,Annsun Boutique,Apollo Serviced Apartments,Apple Residency,Aryaas,Asiana Place,Atchaya,Avnb Towers,Azeez Avenue,BKR Convention Centre,BKR Grand,Beach Park Resort,Bell Hotel Chennai,Benzz Park,Beverly Hotel,Bhimaas,Bluten by Traditions Inn,Brownstar Boutique Hotel,Buhari Royale,Chandra Park,Chennai Palace,Clarion Hotel President,...,The Chariot,The Checkers,The Gateway IT Expressway,The Iris,The Kings,The Leela Palace,The Lotus,The Lotus Sameera,The Lotus Serviced Apartments,The Mount Manor,The Park Chennai,The Pride,The Raintree,"The Raintree, St. Mary's Road",The Residency Chennai,The Residency Towers,The Royal Regency,The Spin,The Spring,The Vijay Park-Arumbakkam,The Westin Chennai Velachery,Thulashi Park,Treebo Angson,Treebo Elan,Treebo Grand Marena,Treebo Hive,Treebo J's Five Two Boutique,Treebo Nestlay Casa,Treebo Saravana Boutique Inn,Tri Sea Residency,Trident Chennai,Turyaa Chennai,VGP Golden Beach Resort,Vestin Park,Victoria,Yashwanth Residency,Zama Lodge,Zone By The Park Orr Chennai,ibis Chennai City Centre,ibis Chennai Sipcot
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,Park Hyatt,Refuge in Chennai,Excellent room and exercise facility. All arou...,80.0,excel room exercis facil around atmospher calm...,"{'neg': 0.071, 'neu': 0.581, 'pos': 0.348, 'co...",0.071,0.581,0.348,0.8225,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hilton Chennai,Hilton Chennai,Very comfortable and felt safe. \r\nStaff were...,100.0,comfort felt safe staff help respect breakfast...,"{'neg': 0.0, 'neu': 0.537, 'pos': 0.463, 'comp...",0.000,0.537,0.463,0.8807,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,The Royal Regency,No worth the rating shown in websites. Pricing...,Not worth the rating shown. Service is not goo...,71.0,worth rate show servic good room well maintain...,"{'neg': 0.077, 'neu': 0.496, 'pos': 0.427, 'co...",0.077,0.496,0.427,0.9022,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Rivera,Good stay,"First of all nice & courteous staff, only one ...",86.0,first nice courteous staff one con stay time c...,"{'neg': 0.094, 'neu': 0.59, 'pos': 0.316, 'com...",0.094,0.590,0.316,0.7506,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Park Hyatt,Needs improvement,Overall ambience of the hotel is very good. In...,86.0,overal ambienc hotel good room facil need impr...,"{'neg': 0.0, 'neu': 0.884, 'pos': 0.116, 'comp...",0.000,0.884,0.116,0.4404,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2346,Hyatt Regency Chennai,Average,Most impressive service by staff in all areas....,80.0,impress servic staff area good restaur fit cen...,"{'neg': 0.0, 'neu': 0.57, 'pos': 0.43, 'compou...",0.000,0.570,0.430,0.9678,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2347,New Woodlands,Homely villa,New woodlands chennai which gave me a

In [ ]:
data["sentiments_title"] = data["Review_Title"].apply(lambda x: sid.polarity_scores(x))
data

,Hotel_name,Review_Title,Review_Text,Rating,Review_clean,sentiments,sentiments_neg,sentiments_neu,sentiments_pos,sentiments_compound,Aadithya,Abu sarovar portico ex Abu Palace,Accord Metropolitan,Akash Inn,Akshay Inn,Aloft Chennai Omr - It Expressway,Ambica Empire,Ammu Palace,Ampa Skywalk,Annsun Boutique,Apollo Serviced Apartments,Apple Residency,Aryaas,Asiana Place,Atchaya,Avnb Towers,Azeez Avenue,BKR Convention Centre,BKR Grand,Beach Park Resort,Bell Hotel Chennai,Benzz Park,Beverly Hotel,Bhimaas,Bluten by Traditions Inn,Brownstar Boutique Hotel,Buhari Royale,Chandra Park,Chennai Palace,Clarion Hotel President,...,The Checkers,The Gateway IT Expressway,The Iris,The Kings,The Leela Palace,The Lotus,The Lotus Sameera,The Lotus Serviced Apartments,The Mount Manor,The Park Chennai,The Pride,The Raintree,"The Raintree, St. Mary's Road",The Residency Chennai,The Residency Towers,The Royal Regency,The Spin,The Spring,The Vijay Park-Arumbakkam,The Westin Chennai Velachery,Thulashi Park,Treebo Angson,Treebo Elan,Treebo Grand Marena,Treebo Hive,Treebo J's Five Two Boutique,Treebo Nestlay Casa,Treebo Saravana Boutique Inn,Tri Sea Residency,Trident Chennai,Turyaa Chennai,VGP Golden Beach Resort,Vestin Park,Victoria,Yashwanth Residency,Zama Lodge,Zone By The Park Orr Chennai,ibis Chennai City Centre,ibis Chennai Sipcot,sentiments_title
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,Park Hyatt,Refuge in Chennai,Excellent room and exercise facility. All arou...,80.0,excel room exercis facil around atmospher calm...,"{'neg': 0.071, 'neu': 0.581, 'pos': 0.348, 'co...",0.071,0.581,0.348,0.8225,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1,Hilton Chennai,Hilton Chennai,Very comfortable and felt safe. \r\nStaff were...,100.0,comfort felt safe staff help respect breakfast...,"{'neg': 0.0, 'neu': 0.537, 'pos': 0.463, 'comp...",0.000,0.537,0.463,0.8807,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,The Royal Regency,No worth the rating shown in websites. Pricing...,Not worth the rating shown. Service is not goo...,71.0,worth rate show servic good room well maintain...,"{'neg': 0.077, 'neu': 0.496, 'pos': 0.427, 'co...",0.077,0.496,0.427,0.9022,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"{'neg': 0.157, 'neu': 0.499, 'pos': 0.344, 'co..."
3,Rivera,Good stay,"First of all nice & courteous staff, only one ...",86.0,first nice courteous staff one con stay time c...,"{'neg': 0.094, 'neu': 0.59, 'pos': 0.316, 'com...",0.094,0.590,0.316,0.7506,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"{'neg': 0.0, 'neu': 0.256, 'pos': 0.744, 'comp..."
4,Park Hyatt,Needs improvement,Overall ambience of the hotel is very good. In...,86.0,overal ambienc hotel good room facil need impr...,"{'neg': 0.0, 'neu': 0.884, 'pos': 0.116, 'comp...",0.000,0.884,0.116,0.4404,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"{'neg': 0.0, 'neu': 0.25, 'pos': 0.75, 'compou..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2346,Hyatt Regency Chennai,Average,Most impressive service by staff in all areas....,80.0,impress servic staff area good restaur fit cen...,"{'neg': 0.0, 'neu': 0.57, 'pos': 0

In [ ]:
data['sentiments_title_neg'] = data['sentiments_title'].apply(lambda x: x['neg'])
data['sentiments_title_neu'] = data['sentiments_title'].apply(lambda x: x['neu'])
data['sentiments_title_pos'] = data['sentiments_title'].apply(lambda x: x['pos'])
data['sentiments_title_compound'] = data['sentiments_title'].apply(lambda x: x['compound'])
data

,Hotel_name,Review_Title,Review_Text,Rating,Review_clean,sentiments,sentiments_neg,sentiments_neu,sentiments_pos,sentiments_compound,Aadithya,Abu sarovar portico ex Abu Palace,Accord Metropolitan,Akash Inn,Akshay Inn,Aloft Chennai Omr - It Expressway,Ambica Empire,Ammu Palace,Ampa Skywalk,Annsun Boutique,Apollo Serviced Apartments,Apple Residency,Aryaas,Asiana Place,Atchaya,Avnb Towers,Azeez Avenue,BKR Convention Centre,BKR Grand,Beach Park Resort,Bell Hotel Chennai,Benzz Park,Beverly Hotel,Bhimaas,Bluten by Traditions Inn,Brownstar Boutique Hotel,Buhari Royale,Chandra Park,Chennai Palace,Clarion Hotel President,...,The Leela Palace,The Lotus,The Lotus Sameera,The Lotus Serviced Apartments,The Mount Manor,The Park Chennai,The Pride,The Raintree,"The Raintree, St. Mary's Road",The Residency Chennai,The Residency Towers,The Royal Regency,The Spin,The Spring,The Vijay Park-Arumbakkam,The Westin Chennai Velachery,Thulashi Park,Treebo Angson,Treebo Elan,Treebo Grand Marena,Treebo Hive,Treebo J's Five Two Boutique,Treebo Nestlay Casa,Treebo Saravana Boutique Inn,Tri Sea Residency,Trident Chennai,Turyaa Chennai,VGP Golden Beach Resort,Vestin Park,Victoria,Yashwanth Residency,Zama Lodge,Zone By The Park Orr Chennai,ibis Chennai City Centre,ibis Chennai Sipcot,sentiments_title,sentiments_title_neg,sentiments_title_neu,sentiments_title_pos,sentiments_title_compound
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,Park Hyatt,Refuge in Chennai,Excellent room and exercise facility. All arou...,80.0,excel room exercis facil around atmospher calm...,"{'neg': 0.071, 'neu': 0.581, 'pos': 0.348, 'co...",0.071,0.581,0.348,0.8225,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
1,Hilton Chennai,Hilton Chennai,Very comfortable and felt safe. \r\nStaff were...,100.0,comfort felt safe staff help respect breakfast...,"{'neg': 0.0, 'neu': 0.537, 'pos': 0.463, 'comp...",0.000,0.537,0.463,0.8807,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
2,The Royal Regency,No worth the rating shown in websites. Pricing...,Not worth the rating shown. Service is not goo...,71.0,worth rate show servic good room well maintain...,"{'neg': 0.077, 'neu': 0.496, 'pos': 0.427, 'co...",0.077,0.496,0.427,0.9022,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"{'neg': 0.157, 'neu': 0.499, 'pos': 0.344, 'co...",0.157,0.499,0.344,0.3885
3,Rivera,Good stay,"First of all nice & courteous staff, only one ...",86.0,first nice courteous staff one con stay time c...,"{'neg': 0.094, 'neu': 0.59, 'pos': 0.316, 'com...",0.094,0.590,0.316,0.7506,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"{'neg': 0.0, 'neu': 0.256, 'pos': 0.744, 'comp...",0.000,0.256,0.744,0.4404
4,Park Hyatt,Needs improvement,Overall ambience of the hotel is very good. In...,86.0,overal ambienc hotel good room facil need impr...,"{'neg': 0.0, 'neu': 0.884, 'pos': 0.116, 'comp...",0.000,0.884,0.116,0.4404,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"{'neg': 0.0, 'neu': 0.25, 'pos': 0.75, 'compou...",0.000,0.250,0.750,0.4588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2346,Hyatt Regency Chennai,Average,Most impressive service 

In [ ]:
df = data.drop(['Hotel_name', 'Review_Title', 'Review_Text', 'sentiments', 'sentiments_title'], axis=1)
df

,Rating,Review_clean,sentiments_neg,sentiments_neu,sentiments_pos,sentiments_compound,Aadithya,Abu sarovar portico ex Abu Palace,Accord Metropolitan,Akash Inn,Akshay Inn,Aloft Chennai Omr - It Expressway,Ambica Empire,Ammu Palace,Ampa Skywalk,Annsun Boutique,Apollo Serviced Apartments,Apple Residency,Aryaas,Asiana Place,Atchaya,Avnb Towers,Azeez Avenue,BKR Convention Centre,BKR Grand,Beach Park Resort,Bell Hotel Chennai,Benzz Park,Beverly Hotel,Bhimaas,Bluten by Traditions Inn,Brownstar Boutique Hotel,Buhari Royale,Chandra Park,Chennai Palace,Clarion Hotel President,Cloud Nine Serviced Apartments,Courtyard Chennai India,Crescent Hospitality Chennai,Crescent Park,...,The Kings,The Leela Palace,The Lotus,The Lotus Sameera,The Lotus Serviced Apartments,The Mount Manor,The Park Chennai,The Pride,The Raintree,"The Raintree, St. Mary's Road",The Residency Chennai,The Residency Towers,The Royal Regency,The Spin,The Spring,The Vijay Park-Arumbakkam,The Westin Chennai Velachery,Thulashi Park,Treebo Angson,Treebo Elan,Treebo Grand Marena,Treebo Hive,Treebo J's Five Two Boutique,Treebo Nestlay Casa,Treebo Saravana Boutique Inn,Tri Sea Residency,Trident Chennai,Turyaa Chennai,VGP Golden Beach Resort,Vestin Park,Victoria,Yashwanth Residency,Zama Lodge,Zone By The Park Orr Chennai,ibis Chennai City Centre,ibis Chennai Sipcot,sentiments_title_neg,sentiments_title_neu,sentiments_title_pos,sentiments_title_compound
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,80.0,excel room exercis facil around atmospher calm...,0.071,0.581,0.348,0.8225,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,1.000,0.000,0.0000
1,100.0,comfort felt safe staff help respect breakfast...,0.000,0.537,0.463,0.8807,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,1.000,0.000,0.0000
2,71.0,worth rate show servic good room well maintain...,0.077,0.496,0.427,0.9022,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.157,0.499,0.344,0.3885
3,86.0,first nice courteous staff one con stay time c...,0.094,0.590,0.316,0.7506,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0.256,0.744,0.4404
4,86.0,overal ambienc hotel good room facil need impr...,0.000,0.884,0.116,0.4404,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0.250,0.750,0.4588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2346,80.0,impress servic staff area good restaur fit cen...,0.000,0.570,0.430,0.9678,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,1.000,0.000,0.0000
2347,71.0,new woodland chennai give home experi luxuri t...,0.000,0.811,0.189,0.7717,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,1.000,0.000,0.0000
2348,100.0,awesom like neat mainten facil reason price ov...,0.000,0.515,0.485,0.9042,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0.517,0.483,0.4215


Попробуем применить на чистом тексте TF-IDF.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
idf_vectorizer = TfidfVectorizer()

Y = idf_vectorizer.fit_transform(df['Review_clean'])
X = Y.todense()
X

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
y = df.Rating
y

Id
0        80.0
1       100.0
2        71.0
3        86.0
4        86.0
        ...  
2346     80.0
2347     71.0
2348    100.0
2349     80.0
2350     71.0
Name: Rating, Length: 2351, dtype: float64

Сравним разные модели:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

knn = KNeighborsClassifier()
cross_val_score(knn, X, y, cv=5).mean()

0.21651352938519217

In [ ]:
from sklearn.svm import SVC

clf = SVC(gamma='auto')
cross_val_score(clf, X, y, cv=5).mean()

0.19778831820029813

In [ ]:
from sklearn.tree import DecisionTreeRegressor

clf = DecisionTreeRegressor()
cross_val_score(clf, X, y, cv=5).mean()

0.184189035934267

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
cross_val_score(clf, X, y, cv=5).mean()

0.44704792880697475

In [ ]:
from sklearn.ensemble import RandomForestRegressor

clf = RandomForestRegressor()
cross_val_score(clf, X, y, cv=5).mean()

0.5054567652435732

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

estimator = RandomForestRegressor(random_state=42)
param_grid = { 
            "n_estimators"      : [10,20,30],
            "max_features"      : ["auto", "sqrt", "log2"],
            "min_samples_split" : [2,4,8],
            "bootstrap": [True, False],
            }


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

grid = GridSearchCV(estimator, param_grid, cv=5)
grid.fit(X_train, y_train)
grid.best_score_

0.5166921895129896

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
grid.best_params_

{'bootstrap': False,
 'max_features': 'sqrt',
 'min_samples_split': 2,
 'n_estimators': 30}

Мы разделили нашу выборку на трейн и тест. Обучим на трейне наш PCA, который преобразует наш вектор слов в вектор поменьше. Затем применим наилучший регрессор.

In [ ]:
from sklearn.decomposition import PCA

res = []

for i in np.arange(300, 400):
  model = PCA(n_components=i)
  X_pca = model.fit(X_train).transform(X_train)
  res_i = cross_val_score(LogisticRegression(random_state=42), X_pca, y_train, scoring='accuracy').mean()
  res.append((i, res_i))

res

In [ ]:
n = 95 # Почему так - см. ячейку выше

pca = PCA(n_components=n)
X_pca = pca.fit(X).transform(X)

In [ ]:
X_df = pd.DataFrame(X_pca, columns=np.arange(X_pca.shape[1]))
X_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94
0,-0.093731,-0.046945,0.021526,0.025010,0.056080,0.091742,-0.005451,0.062356,-0.043574,0.059693,0.123226,0.206443,-0.026983,0.064562,-0.094397,-0.060456,0.042024,-0.045049,0.039639,-0.041099,-0.071046,-0.049331,-0.038609,0.005354,0.017942,-0.029605,-0.000951,0.007688,0.009360,0.001960,-0.017303,-0.109332,-0.013783,-0.049364,-0.025817,-0.006892,-0.007411,-0.016624,-0.040220,0.001336,...,-0.009608,0.028554,0.032054,0.003794,0.019590,0.002223,-0.027277,0.070457,-0.035141,-0.001978,0.000338,0.037184,-0.013937,-0.029165,-0.040532,0.023210,0.006860,-0.000159,0.028262,-0.006727,-0.001026,-0.004617,0.079741,0.007933,-0.022657,-0.074627,-0.054565,0.074908,-0.046126,0.027209,0.050366,0.096684,-0.042363,-0.001386,0.010088,0.035138,0.019140,-0.011623,-0.009283,-0.070113
1,-0.130708,-0.087067,0.049778,-0.025429,-0.078740,0.094604,0.123168,-0.056321,-0.102056,0.006369,0.027200,-0.039390,0.062356,0.000410,0.045804,-0.062972,0.042138,0.031205,0.004641,-0.035987,0.025506,-0.013799,0.002736,-0.014434,-0.003684,-0.017470,-0.021461,-0.037453,0.012665,-0.021193,-0.014047,-0.013613,-0.032513,0.007712,-0.020504,-0.007436,-0.048854,0.008539,0.000713,0.032624,...,-0.028066,0.005005,0.036036,-0.069927,0.046588,0.031074,0.061218,-0.024075,0.015600,-0.011641,-0.026596,0.024107,-0.040905,0.007993,-0.021727,-0.025579,-0.107920,0.016113,-0.083172,-0.034779,0.038375,-0.020608,-0.019717,-0.003024,0.013417,-0.041599,-0.053987,0.049950,0.035230,-0.040297,0.058291,0.019842,-0.005245,0.016981,0.023077,0.015050,-0.024355,-0.064742,-0.042256,-0.009042
2,0.033666,-0.081612,-0.050963,0.026376,0.074645,0.057977,-0.139377,-0.014880,0.002158,0.093033,0.028653,0.112974,-0.025416,-0.035639,-0.070049,-0.086946,-0.009950,-0.024078,-0.029646,0.059049,0.026852,0.065432,-0.033640,0.030884,0.147242,0.070468,0.043499,0.050067,-0.025263,-0.007098,0.117483,0.079817,0.071935,-0.022580,-0.050814,0.102750,0.101564,-0.001870,-0.045590,-0.048709,...,-0.048492,0.014101,0.006871,-0.019728,0.029547,-0.009110,-0.061411,0.014663,0.004212,0.016403,-0.040912,0.018408,-0.047856,0.020061,0.016570,-0.022713,-0.009464,-0.048332,-0.036735,-0.006844,-0.012005,0.027768,-0.055244,0.026537,0.072362,-0.034032,0.015749,-0.007694,-0.005768,0.028372,-0.008744,-0.007876,0.004504,-0.040738,-0.015315,-0.030875,0.080621,0.041919,-0.018707,-0.008911
3,-0.095194,0.025727,0.058711,-0.048190,0.059948,0.008733,0.028606,0.004876,0.096008,0.050817,-0.045417,-0.073991,-0.158089,-0.071974,-0.104110,-0.004799,0.063191,0.124279,0.039651,-0.039015,0.000482,0.027462,0.003416,-0.023281,-0.009352,0.052438,-0.029498,0.024229,0.134202,-0.034155,0.096893,-0.035063,-0.073924,0.036443,0.043536,-0.106382,0.023481,0.112191,-0.014274,0.037928,...,-0.067241,-0.089895,-0.006142,0.021943,-0.053369,0.055588,0.088274,-0.034020,0.053583,0.000096,0.033615,0.018224,-0.052212,-0.055533,-0.039533,0.003569,-0.006170,0.059103,0.008857,-0.024017,0.077687,0.011461,0.005316,0.055012,-0.019292,0.060767,0.019689,-0.037590,0.077130,-0.001881,0.070540,0.004045,-0.040627,-0.016049,-0.036403,-0.037222,-0.074729,0.007556,0.096052,-0.040204
4,0.031048,0.032445,-0.155233,-0.011979,0.084838,0.120263,-0.025518,0.063072,0.120997,0.019161,0.328295,-0.106875,-0.001286,0.182148,0.217637,0.165114,0.111276,-0.121300,0.109635,-0.062680,-0.147800,-0.043885,0.050088,-0.018359,0.097447,-0.145744,0.023166,-0.060755,-0.082288,0.105549,0.067397,0.049674,0.020862,-0.067066,-0.046930,-0.070366,-0.052001,-0.126734,-0.089686,-0.000171,...,-0.002807,0.020294,0.131512,-0.031382,-0.032722,-0.094280,0.007239,-0.053932,0.064849,-0.056049,-0.030707,0.020939,0.021282,-0.018751,-0.015049,0.011333,0.057776,0.068588,0.048563,0.116783,-0.016436,0.066883,-0.007437,0.033932,-0.030250,0.044360,0.015867,0.019710,-0.043431,0.037047,-0.030073,-0.010457,0.00759

In [ ]:
dataset = df.drop(['Rating', 'Review_clean'], axis=1).join(X_df)
dataset

,sentiments_neg,sentiments_neu,sentiments_pos,sentiments_compound,Aadithya,Abu sarovar portico ex Abu Palace,Accord Metropolitan,Akash Inn,Akshay Inn,Aloft Chennai Omr - It Expressway,Ambica Empire,Ammu Palace,Ampa Skywalk,Annsun Boutique,Apollo Serviced Apartments,Apple Residency,Aryaas,Asiana Place,Atchaya,Avnb Towers,Azeez Avenue,BKR Convention Centre,BKR Grand,Beach Park Resort,Bell Hotel Chennai,Benzz Park,Beverly Hotel,Bhimaas,Bluten by Traditions Inn,Brownstar Boutique Hotel,Buhari Royale,Chandra Park,Chennai Palace,Clarion Hotel President,Cloud Nine Serviced Apartments,Courtyard Chennai India,Crescent Hospitality Chennai,Crescent Park,Crystal Residency,Deccan Plaza,...,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.071,0.581,0.348,0.8225,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.009608,0.028554,0.032054,0.003794,0.019590,0.002223,-0.027277,0.070457,-0.035141,-0.001978,0.000338,0.037184,-0.013937,-0.029165,-0.040532,0.023210,0.006860,-0.000159,0.028262,-0.006727,-0.001026,-0.004617,0.079741,0.007933,-0.022657,-0.074627,-0.054565,0.074908,-0.046126,0.027209,0.050366,0.096684,-0.042363,-0.001386,0.010088,0.035138,0.019140,-0.011623,-0.009283,-0.070113
1,0.000,0.537,0.463,0.8807,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.028066,0.005005,0.036036,-0.069927,0.046588,0.031074,0.061218,-0.024075,0.015600,-0.011641,-0.026596,0.024107,-0.040905,0.007993,-0.021727,-0.025579,-0.107920,0.016113,-0.083172,-0.034779,0.038375,-0.020608,-0.019717,-0.003024,0.013417,-0.041599,-0.053987,0.049950,0.035230,-0.040297,0.058291,0.019842,-0.005245,0.016981,0.023077,0.015050,-0.024355,-0.064742,-0.042256,-0.009042
2,0.077,0.496,0.427,0.9022,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.048492,0.014101,0.006871,-0.019728,0.029547,-0.009110,-0.061411,0.014663,0.004212,0.016403,-0.040912,0.018408,-0.047856,0.020061,0.016570,-0.022713,-0.009464,-0.048332,-0.036735,-0.006844,-0.012005,0.027768,-0.055244,0.026537,0.072362,-0.034032,0.015749,-0.007694,-0.005768,0.028372,-0.008744,-0.007876,0.004504,-0.040738,-0.015315,-0.030875,0.080621,0.041919,-0.018707,-0.008911
3,0.094,0.590,0.316,0.7506,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.067241,-0.089895,-0.006142,0.021943,-0.053369,0.055588,0.088274,-0.034020,0.053583,0.000096,0.033615,0.018224,-0.052212,-0.055533,-0.039533,0.003569,-0.006170,0.059103,0.008857,-0.024017,0.077687,0.011461,0.005316,0.055012,-0.019292,0.060767,0.019689,-0.037590,0.077130,-0.001881,0.070540,0.004045,-0.040627,-0.016049,-0.036403,-0.037222,-0.074729,0.007556,0.096052,-0.040204
4,0.000,0.884,0.116,0.4404,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.002807,0.020294,0.131512,-0.031382,-0.032722,-0.094280,0.007239,-0.053932,0.064849,-0.056049,-0.030707,0.020939,0.021282,-0.018751,-0.015049,0.011333,0.057776,0.068588,0.048563,0.116783,-0.016436,0.066883,-0.007437,0.033932,-0.030250,0.044360,0.015867,0.019710,-0.043431,0.037047,-0.030073,-0.010457,0.007590,0.006686,-0.009989,-0.029282,-0.054756,-0.030457,0.099329,0.031464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2346,0.000,0.570,0.430,0.9678,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.021379,-0.080242,0.025919,0.019098,-0.054671,0.056374,0.118285,-0.023677,-0.012645,0.122367,0.046530,-0.089570,0.014056,-0.061078,-0.070167,0.119049,0.030197,-0.048424,0.029706,0.054940,0.014987,0.013875,0.034692,-0.010450,-0.078915,-0.036425,0.056104,-0.

In [ ]:
RFR = RandomForestRegressor(n_estimators=30, min_samples_split=2, max_features='sqrt', bootstrap=False, random_state=42)
cross_val_score(RFR, dataset, y, cv=5).mean()

0.6203250530871897

In [ ]:
-cross_val_score(RFR, dataset, y, cv=5, scoring='neg_root_mean_squared_error').mean()

12.990932541047533

In [ ]:
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(dataset, y, test_size=0.2, random_state=42)

In [ ]:
!pip install catboost

     |████████████████████████████████| 67.3MB 62kB/s 


In [ ]:
from catboost import CatBoostRegressor

cbr = CatBoostRegressor(random_state=42)
cbr.fit(X_train, y_train)
y_pred_cbr = cbr.predict(X_test)

mean_squared_error(y_test, y_pred_cbr)

Learning rate set to 0.043672
0:	learn: 20.7763873	total: 37.9ms	remaining: 37.9s
1:	learn: 20.4744083	total: 74ms	remaining: 36.9s
2:	learn: 20.1761696	total: 113ms	remaining: 37.6s
3:	learn: 19.8860143	total: 154ms	remaining: 38.4s
4:	learn: 19.6207576	total: 192ms	remaining: 38.3s
5:	learn: 19.3504768	total: 236ms	remaining: 39.1s
6:	learn: 19.1060076	total: 276ms	remaining: 39.1s
7:	learn: 18.8476258	total: 316ms	remaining: 39.2s
8:	learn: 18.5989882	total: 353ms	remaining: 38.8s
9:	learn: 18.3643060	total: 390ms	remaining: 38.6s
10:	learn: 18.1653712	total: 428ms	remaining: 38.5s
11:	learn: 17.9888412	total: 488ms	remaining: 40.2s
12:	learn: 17.8036664	total: 526ms	remaining: 39.9s
13:	learn: 17.6629872	total: 565ms	remaining: 39.8s
14:	learn: 17.4959688	total: 605ms	remaining: 39.7s
15:	learn: 17.3349237	total: 649ms	remaining: 39.9s
16:	learn: 17.1917796	total: 689ms	remaining: 39.8s
17:	learn: 17.0444415	total: 730ms	remaining: 39.8s
18:	learn: 16.9028004	total: 769ms	remaining

160.595386342698

In [ ]:
mean_squared_error(y_test, y_pred_cbr, squared=False)

12.672623498814206

Получили по метрике RMSE 12.67.